In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/spam-email/spam.csv


In [2]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

In [3]:
df = pd.read_csv('/kaggle/input/spam-email/spam.csv')
df

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [4]:
df.duplicated()

0       False
1       False
2       False
3       False
4       False
        ...  
5567    False
5568    False
5569    False
5570    False
5571    False
Length: 5572, dtype: bool

In [5]:
from sklearn.preprocessing import OneHotEncoder
ohe =  OneHotEncoder(sparse = False)
df_new = ohe.fit_transform(df[['Category']])
df_new = pd.DataFrame(df_new, columns=ohe.get_feature_names_out(['Category'])).astype(int)
merged_df = pd.concat([df, df_new], axis='columns')
merged_df

/opt/conda/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


,Category,Message,Category_ham,Category_spam
0,ham,"Go until jurong point, crazy.. Available only ...",1,0
1,ham,Ok lar... Joking wif u oni...,1,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,0,1
3,ham,U dun say so early hor... U c already then say...,1,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",1,0
...,...,...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...,0,1
5568,ham,Will ü b going to esplanade fr home?,1,0
5569,ham,"Pity, * was in mood for that. So...any other s...",1,0
5570,ham,The guy did some bitching but I acted like i'd...,1,0


In [6]:
final_df = merged_df.drop(['Category','Category_ham'], axis = 'columns')
final_df

,Message,Category_spam
0,"Go until jurong point, crazy.. Available only ...",0
1,Ok lar... Joking wif u oni...,0
2,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,U dun say so early hor... U c already then say...,0
4,"Nah I don't think he goes to usf, he lives aro...",0
...,...,...
5567,This is the 2nd time we have tried 2 contact u...,1
5568,Will ü b going to esplanade fr home?,0
5569,"Pity, * was in mood for that. So...any other s...",0
5570,The guy did some bitching but I acted like i'd...,0


In [7]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(final_df.Message,final_df.Category_spam, test_size=0.3)

In [8]:
from sklearn.pipeline import Pipeline
clf = Pipeline([('vectorizer', CountVectorizer()),
               ('nb', MultinomialNB())
               ])

In [9]:
clf.fit(x_train,y_train)

Pipeline(steps=[('vectorizer', CountVectorizer()), ('nb', MultinomialNB())])

In [10]:
y_pred = clf.predict(x_test)
y_pred

array([0, 1, 0, ..., 0, 0, 0])

In [11]:
clf.score(x_test,y_test)

0.9886363636363636

In [12]:
from sklearn.metrics import confusion_matrix, classification_report

In [13]:
cm= confusion_matrix(y_test, y_pred)
cr = classification_report(y_test, y_pred)

In [14]:
cm

array([[1445,    2],
       [  17,  208]])

In [15]:
cr

'              precision    recall  f1-score   support\n\n           0       0.99      1.00      0.99      1447\n           1       0.99      0.92      0.96       225\n\n    accuracy                           0.99      1672\n   macro avg       0.99      0.96      0.97      1672\nweighted avg       0.99      0.99      0.99      1672\n'

In [19]:
import pandas as pd

lines = cr.strip().split('\n')

headers = ["class", "precision", "recall", "f1-score", "support"]

data = []
for line in lines[1:]:
    row = line.split()
    if len(row) > 5:
        row = [' '.join(row[:-4])] + row[-4:]
    data.append(row)

df = pd.DataFrame(data, columns=headers)

print(df)


          class precision recall f1-score support
0          None      None   None     None    None
1             0      0.99   1.00     0.99    1447
2             1      0.99   0.92     0.96     225
3          None      None   None     None    None
4      accuracy      0.99   1672     None    None
5     macro avg      0.99   0.96     0.97    1672
6  weighted avg      0.99   0.99     0.99    1672
